In [1]:
#export
"""
This is for quick modifiers, think of them as changing formats
"""
__all__ = ["apply", "applySingle", "lstrip", "rstrip", "strip",
           "upper", "lower", "replace", "remove", "toFloat", "toInt", "sort"]
from typing import Callable, Iterator, Any
from k1lib.bioinfo.cli.init import patchDefaultDelim, BaseCli, settings
import k1lib.bioinfo.cli as cli

In [2]:
#export
class apply(BaseCli):
    def __init__(self, f:Callable[[str], str], column:int=None):
        """Applies a function f to every line

:param column: if not None, then applies the function to that column only"""
        self.f = f; self.column = column
    def __ror__(self, it:Iterator[str]):
        f = self.f; c = self.column
        if c is None: return (f(line) for line in it)
        else: return (((e if i != c else f(e)) 
                       for i, e in enumerate(row)) for row in it)
class applySingle(BaseCli):
    def __init__(self, f:Callable[[Any], Any]):
        """Like :class:`apply`, but much simpler, just operating on the entire input
object, essentially"""
        self.f = f
    def __ror__(self, it:Any) -> Any: return self.f(it)
def lstrip(column:int=None, char:str=None):
    """Strips left of every line"""
    return apply(lambda e: e.lstrip(char), column)
def rstrip(column:int=None, char:str=None):
    """Strips right of every line"""
    return apply(lambda e: e.rstrip(char), column)
def strip(column:int=None, char:str=None):
    """Strips both sides of every line"""
    return apply(lambda e: e.strip(char), column)

In [3]:
#export
def upper(column:int=None):
    """Make all characters uppercase"""
    return apply(lambda e: e.upper(), column)
def lower(column:int=None):
    """Make all characters lowercase"""
    return apply(lambda e: e.lower(), column)

In [4]:
#export
def replace(s:str, target:str=None, column:int=None):
    """Replaces substring `s` with `target` for each line."""
    t = patchDefaultDelim(target)
    return apply(lambda e: e.replace(s, t), column)
def remove(s:str, column:int=None):
    """Removes a specific substring in each line."""
    return replace(s, "", column)

In [5]:
#export
def toFloat(column:int=None):
    """Converts every row into a float. Excludes non numbers if not in
:ref:`strict mode <bioinfoSettings>`."""
    f = apply(lambda e: float(e), column)
    return f if settings["strict"] else cli.isNumeric(column) | f
def toInt(column:int=None):
    """Converts every row into an integer. Excludes non numbers if not in
:ref:`strict mode <bioinfoSettings>`."""
    f = apply(lambda e: int(float(e)), column)
    return f if settings["strict"] else cli.isNumeric(column) | f

In [6]:
#export
class sort(BaseCli):
    def __init__(self, column:int=0, numeric=True, reverse=False):
        """Sorts all lines based on a specific `column`.

:param numeric: whether to treat column as float
:param reverse: False for smaller to bigger, True for bigger to smaller. Use
    :meth:`__invert__` to quickly reverse the order instead of using this param"""
        self.column = column; self.reverse = reverse; self.numeric = numeric
        self.filterF = (lambda x: float(x)) if numeric else (lambda x: x)
    def __ror__(self, it:Iterator[str]):
        c = self.column; f = self.filterF
        rows = list(it | cli.isNumeric(c) if self.numeric else it)
        def sortF(row):
            if len(row) > c: return f(row[c])
            return float("inf")
        return iter(sorted(rows, key=sortF, reverse=self.reverse))
    def __invert__(self):
        """Creates a clone that has the opposite sort order"""
        return sort(self.column, self.numeric, not self.reverse)

In [7]:
!../../../export.py bioinfo/cli/modifier

Current dir: /home/kelvin/repos/labs/k1lib, ../../../export.py
rm: cannot remove '__pycache__': No such file or directory
Found existing installation: k1lib 0.1.10
Uninstalling k1lib-0.1.10:
  Successfully uninstalled k1lib-0.1.10
running install
running bdist_egg
running egg_info
creating k1lib.egg-info
writing k1lib.egg-info/PKG-INFO
writing dependency_links to k1lib.egg-info/dependency_links.txt
writing requirements to k1lib.egg-info/requires.txt
writing top-level names to k1lib.egg-info/top_level.txt
writing manifest file 'k1lib.egg-info/SOURCES.txt'
reading manifest file 'k1lib.egg-info/SOURCES.txt'
writing manifest file 'k1lib.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_lib
running build_py
creating build
creating build/lib
creating build/lib/k1lib
copying k1lib/_learner.py -> build/lib/k1lib
copying k1lib/data.py -> build/lib/k1lib
copying k1lib/selector.py -> build/lib/k1lib
copying k1lib/imports.py -> build/lib/k1lib
copying k1